<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install and Setup (this section will be executed automatically)

#### Install Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-92owidia
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-92owidia
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 0f3a9fbf159542d0392cbbde079d7e812a3fc237
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.3 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.4.0-py3-none-any.whl size=54731 sha256=6ea8f0cf1cefdf491ca9b3e10f823ae9889045d3eadf2dbb4bb16fb90d3a065f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ezxviltt/wheels/07/97/7f/a72c7fbc975be0326ac45fec1ca0c872e2d6e2bbea96caf706
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

In [2]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

#### Setup and Configuration

In [3]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [4]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    dir='ibd'
)

In [5]:
# @title DataFrame Operations
import yfinance as yf
import vistock.tw as tw

def yf_get_current_price(ticker):
    stock = yf.Ticker(ticker)
    historical_data = stock.history(period="1d")
    return historical_data['Close'].iloc[-1]

def is_taiwan_stock_df(df):
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    return ticker.isdigit()

def add_price_column(df):
    column_names = df.columns.tolist()
    if 'Price' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    if df.empty:
        return df
    df['Price'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Price')
    df = df[column_names]
    if is_taiwan_stock_df(df):
        df['Price'] = df['Ticker'].apply(tw.stock_price)
    else:
        df['Price'] = df['Ticker'].apply(yf_get_current_price)
    return df

def add_name_column(df):
    column_names = df.columns.tolist()
    if 'Name' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    if df.empty:
        return df
    if not is_taiwan_stock_df(df):
        return df
    df['Name'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Name')
    df = df[column_names]
    df['Name'] = df['Ticker'].apply(tw.stock_name)
    return df

def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]

In [6]:
# @title OrderedSet

class OrderedSet:
    def __init__(self, iterable=[]):
        self._data = dict.fromkeys(iterable)

    def add(self, item):
        self._data[item] = None

    def discard(self, item):
        self._data.pop(item, None)

    def __contains__(self, item):
        return item in self._data

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return f"OrderedSet({list(self._data)})"

    def union(self, other):
        """Return a new OrderedSet with elements from both sets."""
        combined = OrderedSet(self._data.keys())
        combined._data.update(other._data)
        return combined

    def update(self, other):
        """Update the current OrderedSet with elements from another OrderedSet."""
        self._data.update(other._data)

    def __add__(self, other):
        """Concatenate two OrderedSets."""
        combined = OrderedSet(self._data.keys())
        combined.update(other)
        return combined

### Glossary of Terms

source (The source of stocks to analyze):
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所）
    - `TPEX`: Taipei Exchange （上櫃交易所）
    - `ESB`: Emerging Stock Board （興櫃交易所）
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index （費半指數）
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.


### Execute Actions Step by Step


In [7]:
# @title Step 1. Pick a File

import ipywidgets as widgets

filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[3].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames), reverse=True)
all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
all_periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames to set dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=all_types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=all_periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files), reverse=True)
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update Type dropdown
    available_types = sorted(set(fn.split('_')[1] for fn in date_filtered_files), reverse=True)
    type_dropdown.options = available_types
    if type_dropdown.value not in available_types:
        type_dropdown.value = available_types[0] if available_types else None

    # Update Period dropdown
    available_periods = sorted(set(fn.split('_')[2] for fn in date_filtered_files), reverse=True)
    period_dropdown.options = available_periods
    if period_dropdown.value not in available_periods:
        period_dropdown.value = available_periods[0] if available_periods else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            type_dropdown.value == fn.split('_')[1] and
            period_dropdown.value == fn.split('_')[2])
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown, file_dropdown)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='430px'), options=('20240814', '20240813', '20240812', '2024…

Dropdown(description='Source:', layout=Layout(width='430px'), options=('TWSE+TPEX', 'TWSE', 'SPX+DJIA+NDX+SOX'…

Dropdown(description='Type:', layout=Layout(width='430px'), options=('stocks', 'industries'), value='stocks')

Dropdown(description='Period:', layout=Layout(width='430px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='430px'), options=(), value=None)

In [8]:
# @title Step 2. Print Top Percentile Items {"run":"auto"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
df_top = df.head(num_items).copy()
df_top = add_price_column(df_top)
df_top = add_name_column(df_top)
display(df_top)

if 'Name' in df_top.columns:
    print(list(df_top['Name']))
if 'Ticker' in df_top.columns:
    print(list(df_top['Ticker']))

Source: TWSE
Type: stocks
Period: 2y
Date: 20240814


,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,8374,羅昇,153.0,Industrials,Industrial Distribution,422.13,286.15,86.81,83.23,99.0,99,45,16,1
1,2365,昆盈,64.4,Technology,Computer Hardware,353.22,325.81,133.57,190.34,99.0,99,94,98,2
2,4562,穎漢,79.5,Industrials,Specialty Industrial Machinery,350.95,280.43,147.94,89.24,99.0,99,96,39,3
3,6442,光聖,400.0,Technology,Electronic Components,324.09,478.92,196.89,190.49,99.0,99,99,98,4
4,2543,皇昌,63.8,Industrials,Engineering & Construction,308.50,289.91,273.73,229.39,99.0,99,99,99,5
5,2348,海悅,296.5,Real Estate,Real Estate Services,293.23,243.70,201.33,120.14,99.0,99,99,88,6
6,9906,欣巴巴,218.0,Industrials,Engineering & Construction,283.37,176.80,117.63,112.65,99.0,98,88,82,7
7,2359,所羅門,172.5,Technology,Electronic Components,259.47,249.49,372.62,95.61,99.0,99,99,56,8
8,3535,晶彩科,66.5,Technology,Computer Hardware,248.92,277.36,108.20,108.98,99.0,99,83,79,9
9,2524,京城,124.0,Real Estate,Real Estate—Development,246.73,167.45,130.92,87.21,99.0,97,93,30,10


['羅昇', '昆盈', '穎漢', '光聖', '皇昌', '海悅', '欣巴巴', '所羅門', '晶彩科', '京城']
['8374', '2365', '4562', '6442', '2543', '2348', '9906', '2359', '3535', '2524']


In [9]:
# @title Step 3. Filter Items with Increasing RS > 100 {"run":"auto"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
filtered_df = filter_increasing_relative_strength(df)
df_top_filtered = filtered_df.head(num_items).copy()
df_top_filtered = add_price_column(df_top_filtered)
df_top_filtered = add_name_column(df_top_filtered)
display(df_top_filtered)

if 'Name' in df_top_filtered.columns:
    print(list(df_top_filtered['Name']))
if 'Ticker' in df_top_filtered.columns:
    print(list(df_top_filtered['Ticker']))

Source: TWSE
Type: stocks
Period: 2y
Date: 20240814


,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,8374,羅昇,153.00,Industrials,Industrial Distribution,422.13,286.15,86.81,83.23,99.0,99,45,16,1
2,4562,穎漢,79.50,Industrials,Specialty Industrial Machinery,350.95,280.43,147.94,89.24,99.0,99,96,39,3
4,2543,皇昌,63.80,Industrials,Engineering & Construction,308.50,289.91,273.73,229.39,99.0,99,99,99,5
5,2348,海悅,296.50,Real Estate,Real Estate Services,293.23,243.70,201.33,120.14,99.0,99,99,88,6
6,9906,欣巴巴,218.00,Industrials,Engineering & Construction,283.37,176.80,117.63,112.65,99.0,98,88,82,7
9,2524,京城,124.00,Real Estate,Real Estate—Development,246.73,167.45,130.92,87.21,99.0,97,93,30,10
12,1225,福懋油,159.50,Consumer Defensive,Packaged Foods,225.21,176.00,117.40,88.43,98.0,97,88,35,13
18,8028,昇陽半導體,133.50,Technology,Semiconductors,187.88,156.71,89.72,83.64,98.0,97,53,17,19
21,6215,和椿,67.80,Industrials,Specialty Industrial Machinery,177.82,156.96,156.72,99.71,98.0,97,97,65,22
24,1540,喬福,37.95,Industrials,Specialty Industrial Machinery,173.39,138.39,97.52,92.60,97.0,95,69,49,25


['羅昇', '穎漢', '皇昌', '海悅', '欣巴巴', '京城', '福懋油', '昇陽半導體', '和椿', '喬福']
['8374', '4562', '2543', '2348', '9906', '2524', '1225', '8028', '6215', '1540']


In [15]:
# @title Step 4. Show the Comparison of RS Lines {"run":"auto"}
period = "2y" # @param ["1y","2y","5y"]
interval = "1d" # @param ["1d","1wk"]
backend = "Plotly" # @param ["mplfinance","Plotly"]

import ipywidgets as widgets

# Define available stock tickers
tickers = OrderedSet()
if 'Name' in df_top.columns:
    tickers += OrderedSet(df_top['Name'])
elif 'Ticker' in df_top.columns:
    tickers += OrderedSet(df_top['Ticker'])
if 'Name' in df_top_filtered.columns:
    tickers += OrderedSet(df_top_filtered['Name'])
elif 'Ticker' in df_top_filtered.columns:
    tickers += OrderedSet(df_top_filtered['Ticker'])

available_tickers = list(tickers)

# Create checkboxes for stock selection
checkboxes = [
    widgets.Checkbox(value=False, description=ticker,
                     layout=widgets.Layout(width='auto', margin='2px'))
    for ticker in available_tickers
]

# Arrange checkboxes in a grid layout using GridBox
n_cols = 5  # Number of columns in the grid
n_rows = (len(checkboxes) + n_cols - 1) // n_cols  # Number of rows required

# Ensure all rows have the same number of columns
grid_items = [checkboxes[i:i + n_cols]
              for i in range(0, len(checkboxes), n_cols)]

# Fill any remaining cells with dummy widgets
for row in grid_items:
    while len(row) < n_cols:
        row.append(widgets.Label())  # Use Label as placeholder

# Ensure the number of rows is correct
while len(grid_items) < n_rows:
    grid_items.append([widgets.Label()] * n_cols)  # Add empty rows if necessary

# Flatten the grid_items list
flat_grid_items = [item for sublist in grid_items for item in sublist]

checkbox_grid = widgets.GridBox(flat_grid_items, layout=widgets.Layout(
    width='auto',
    grid_template_columns=f'repeat({n_cols}, 1fr)',
    grid_gap='10px'
))

# Button to confirm selection and plot
button = widgets.Button(description="Generate Plot")

# Create final layout combining grid and controls
layout = widgets.VBox([checkbox_grid, button])

# Display UI elements
display(layout)

def get_selected_stocks():
    selected_stocks = [checkbox.description
                       for checkbox in checkboxes if checkbox.value]
    return selected_stocks

def validate_selection(selected_stocks):
    if len(selected_stocks) > 8:
        print("You can only select up to 8 stocks. Please deselect some items.")
        return selected_stocks[:8]  # Limit to 8 items
    return selected_stocks


import vistock.mpl as vs_mpl
import vistock.plotly as vs_plotly


func_dic = {
    'mplfinance': vs_mpl.ibd_rs_cmp,
    'Plotly': vs_plotly.ibd_rs_cmp,
}
ibd_rs_cmp = func_dic[backend]

# Define the button click event
def on_button_click(b):
    selected_stocks = get_selected_stocks()
    symbols = validate_selection(selected_stocks)
    if not symbols:
        print("No stocks selected. Please select at least one stock.")
        return
    ibd_rs_cmp.plot(symbols, interval=interval, period=period)


button.on_click(on_button_click)



[*********************100%%**********************]  2 of 2 completed
